# ETL RAW -> SILVER (PostgreSQL)
**Projeto:** Social Media User Analysis  
**Foco:** Quais características demográficas e de comportamento predizem maior clique em anúncios (`ads_clicked_per_day`)  
**Pipeline:** RAW (bruto) → SILVER (limpo, consistente, pronto para análise/modelagem)


## 1. Contexto do Projeto e Objetivo Analítico
**Este notebook implementa o processo de ETL da camada RAW para SILVER no PostgreSQL.**  
O objetivo do projeto é analisar **quais características demográficas, comportamentais e psicossociais predizem um maior número de cliques em anúncios no Instagram (`ads_clicked_per_day`)**.


## 2. Configuração do Ambiente e Importações
Nesta etapa realizamos a importação das bibliotecas necessárias, definimos os caminhos dos arquivos RAW e configuramos os parâmetros de conexão com o banco de dados PostgreSQL.


In [25]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_batch



INPUT_FILE = '../Data Layer/raw/data_raw.csv'

DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'instagram_usage',
    'user': 'sbd2',
    'password': 'sbd2123'
}

SILVER_SCHEMA = 'silver'
SILVER_TABLE = 'users'

RAW_COLS = [
    "user_id", "age", "gender", "country", "income_level", "employment_status",
    "education_level", "relationship_status", "has_children",
    "perceived_stress_score", "self_reported_happiness", "user_engagement_score",
    "daily_active_minutes_instagram", "time_on_feed_per_day", "time_on_reels_per_day",
    "sessions_per_day", "average_session_length_minutes",
    "ads_viewed_per_day", "ads_clicked_per_day"
]

## 3. Funções Auxiliares de Limpeza e Padronização
Definimos funções reutilizáveis para:
- Normalização de textos categóricos
- Conversão de valores booleanos
- Tratamento de inconsistências textuais
- Remoção de outliers por quantis
Essas funções garantem padronização e reprodutibilidade no pipeline.


In [26]:


def norm_lower(x):
    if pd.isna(x):
        return None
    return str(x).strip().lower()

def to_bool_yesno(x):
    if pd.isna(x):
        return None
    s = str(x).strip().lower()
    if s in {"true"}:
        return True
    if s in {"false"}:
        return False
    return None

def filter_by_quantiles(df, cols, q_low=0.01, q_high=0.99):
    out = df.copy()
    for c in cols:
        if c not in out.columns:
            continue
        lo = out[c].quantile(q_low)
        hi = out[c].quantile(q_high)
        out = out[(out[c] >= lo) & (out[c] <= hi)]
    return out


## 4. Carregamento do arquivo CSV

In [27]:
df = pd.read_csv(INPUT_FILE)

# Seleção e conversão inicial (Em memória)
df = df[[c for c in RAW_COLS if c in df.columns]].copy()
numeric_cols = [
    "age", "perceived_stress_score", "self_reported_happiness", "user_engagement_score",
    "daily_active_minutes_instagram", "time_on_feed_per_day", "time_on_reels_per_day",
    "sessions_per_day", "average_session_length_minutes", "ads_viewed_per_day", "ads_clicked_per_day"
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

## 5. Aplicando transformações

In [30]:
categorical_cols = ['gender', 'country','income_level' 'employment_status', 'education_level', 'relationship_status']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].apply(norm_lower)


if 'has_children' in df.columns:
    df['has_children'] = df['has_children'].apply(to_bool_yesno)

cols_to_filter = ['daily_active_minutes_instagram', 'ads_clicked_per_day', 'average_session_length_minutes']
df = filter_by_quantiles(df, cols_to_filter)

## 6. Carregamento dos Dados Brutos na Camada RAW
Realizamos o carregamento do arquivo CSV original para a tabela `raw.instagram_usage`.  
Os dados são inseridos diretamente no PostgreSQL  seguido de inserção em batch, preservando o caráter bruto da camada RAW.


In [29]:
try:
    with psycopg2.connect(**DB_CONFIG) as conn:
        with conn.cursor() as cur:
            # Garante que o schema existe
            cur.execute(f"CREATE SCHEMA IF NOT EXISTS {SILVER_SCHEMA};")
            
            # Limpa a tabela antes da nova carga (Idempotência)
            cur.execute(f"TRUNCATE TABLE {SILVER_SCHEMA}.{SILVER_TABLE};")
            
            # Prepara a inserção
            cols_list = df.columns.tolist()
            placeholders = ", ".join(["%s"] * len(cols_list))
            colnames = ", ".join(cols_list)
            
            insert_sql = f"INSERT INTO {SILVER_SCHEMA}.{SILVER_TABLE} ({colnames}) VALUES ({placeholders});"
            
            # Converte DataFrame para lista de tuplas para o psycopg2
            data_to_insert = [tuple(x) for x in df.to_numpy()]
            
            execute_batch(cur, insert_sql, data_to_insert, page_size=5000)
            
        conn.commit()
        print(f"Sucesso! {len(data_to_insert)} registros processados e inseridos na Silver.")

except Exception as e:
    print(f"Erro durante o processo de carga: {e}")

Sucesso! 1506286 registros processados e inseridos na Silver.
